# Introduction to Letta using the `LocalClient` 
This notebook is a tutorial on how to use Letta's `LocalClient`. Unlike the `RESTClient` which connects to a running agents service, the `LocalClient` will run agents on your local machine, so does not require connecting to a service. 

This tutorial will cover the basics of creating an agent, interacting with an agent, and understanding the agent's state and memories. 

## Step 0: Install the `letta` package 

In [ ]:
!pip install -U letta

We'll also import a helper function to print out messages from agents in a nice format: 

In [ ]:
import html
import json
import re
from dotenv import find_dotenv, load_dotenv
from IPython.display import HTML, display

def nb_print(messages):
    html_output = """
    <style>
        .message-container {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            max-width: 800px;
            margin: 20px auto;
            background-color: #1e1e1e;
            border-radius: 8px;
            overflow: hidden;
            color: #d4d4d4;
        }
        .message {
            padding: 10px 15px;
            border-bottom: 1px solid #3a3a3a;
        }
        .message:last-child {
            border-bottom: none;
        }
        .title {
            font-weight: bold;
            margin-bottom: 5px;
            color: #ffffff;
            text-transform: uppercase;
            font-size: 0.9em;
        }
        .content {
            background-color: #2d2d2d;
            border-radius: 4px;
            padding: 5px 10px;
            font-family: 'Consolas', 'Courier New', monospace;
            white-space: pre-wrap;
        }
        .status-line {
            margin-bottom: 5px;
            color: #d4d4d4;
        }
        .function-name { color: #569cd6; }
        .json-key { color: #9cdcfe; }
        .json-string { color: #ce9178; }
        .json-number { color: #b5cea8; }
        .json-boolean { color: #569cd6; }
        .internal-monologue { font-style: italic; }
    </style>
    <div class="message-container">
    """

    for msg in messages:
        content = get_formatted_content(msg)

        # don't print empty function returns
        if msg.message_type == "function_return":
            return_data = json.loads(msg.function_return)
            if "message" in return_data and return_data["message"] == "None":
                continue

        title = msg.message_type.replace("_", " ").upper()
        html_output += f"""
        <div class="message">
            <div class="title">{title}</div>
            {content}
        </div>
        """

    html_output += "</div>"
    display(HTML(html_output))


def get_formatted_content(msg):
    if msg.message_type == "internal_monologue":
        return f'<div class="content"><span class="internal-monologue">{html.escape(msg.internal_monologue)}</span></div>'
    elif msg.message_type == "function_call":
        args = format_json(msg.function_call.arguments)
        return f'<div class="content"><span class="function-name">{html.escape(msg.function_call.name)}</span>({args})</div>'
    elif msg.message_type == "function_return":

        return_value = format_json(msg.function_return)
        # return f'<div class="status-line">Status: {html.escape(msg.status)}</div><div class="content">{return_value}</div>'
        return f'<div class="content">{return_value}</div>'
    elif msg.message_type == "user_message":
        if is_json(msg.message):
            return f'<div class="content">{format_json(msg.message)}</div>'
        else:
            return f'<div class="content">{html.escape(msg.message)}</div>'
    elif msg.message_type in ["assistant_message", "system_message"]:
        return f'<div class="content">{html.escape(msg.message)}</div>'
    else:
        return f'<div class="content">{html.escape(str(msg))}</div>'


def is_json(string):
    try:
        json.loads(string)
        return True
    except ValueError:
        return False


def format_json(json_str):
    try:
        parsed = json.loads(json_str)
        formatted = json.dumps(parsed, indent=2, ensure_ascii=False)
        formatted = formatted.replace("&", "&amp;").replace("<", "&lt;").replace(">", "&gt;")
        formatted = formatted.replace("\n", "<br>").replace("  ", "&nbsp;&nbsp;")
        formatted = re.sub(r'(".*?"):', r'<span class="json-key">\1</span>:', formatted)
        formatted = re.sub(r': (".*?")', r': <span class="json-string">\1</span>', formatted)
        formatted = re.sub(r": (\d+)", r': <span class="json-number">\1</span>', formatted)
        formatted = re.sub(r": (true|false)", r': <span class="json-boolean">\1</span>', formatted)
        return formatted
    except json.JSONDecodeError:
        return html.escape(json_str)

## Step 1: Create a `LocalClient` 


In [ ]:
from letta import LocalClient

client = LocalClient()

### Configuring client defaults 
Agents in Letta are model agnostic, so they can connect to different model backends (you can even switch model backends for an existing agents). For this tutorial, we'll set a client default config so that all agents are created with the free letta model endpoints. 

In [ ]:
from letta import LLMConfig, EmbeddingConfig

client.set_default_llm_config(LLMConfig.default_config("letta")) 
client.set_default_embedding_config(EmbeddingConfig.default_config("letta")) 

## Step 2: Creating an agent 

In [ ]:
agent_name = "my_agent"

In [ ]:
from letta.schemas.memory import ChatMemory

agent_state = client.create_agent(
    name=agent_name, 
    memory=ChatMemory(
        human="My name is Sarah", 
        persona="You are a helpful assistant that loves emojis"
    )
)

### Messaging the agent 
Now we can message the agent! This agent will have memories about both itself and the human (you). When we send a message to the agent, we will get back a list of messages from the agents. 

Letta agents have some unique characteristics that allow them to have more advanced reasoning. Notice how: 
* The agent generates *inner thoughts* to think before it acts
* Messages to the user are generated via a `send_message` tool 

In [ ]:
response = client.send_message(
    agent_id=agent_state.id, 
    message="hello!", 
    role="user" 
)
nb_print(response.messages)

## Step 3: Understanding agent state 
Agents are essentailly multi-step reasoning programs which make multiple call to an LLM. Letta manages what is passed to the context window in reach reasoning step. The context window includes: 
* The *system prompt* to define the agent's behavior 
* The set of *tools* the agent has access to 
* The agent's *core memory* (i.e. in-context memory)
* A summary of it's *archival memory* 
* A summary of it's *recall memory* 
* An in-context message queue

In this section, we'll look at the current state of the agent to understand exactly what is being passed to the context window. 

### System Prompt 
The system prompt defines the behavior of the agent. Unlike the memory, the system prompt is not editable. 

In [ ]:
print(agent_state.system)

### Tools 
The agent has access to a set of tools. Each tool is stored in a database, so it can be loaded and executed by the server. Letta also includes a set of default memory management tools, as well as the `send_message` tool to communicate with the human. 

In [ ]:
agent_state.tools

In [ ]:
client.get_tool(client.get_tool_id('send_message'))

### Core memory 
The core memory is the part of memory that is places *in-context*. Core memory is divided into multiple blocks, which each have a `label` and `limit` (the number of characters allocated to storing memories in that block). 

In [ ]:
memory = client.get_core_memory(agent_state.id)

In [ ]:
memory

In [ ]:
memory.get_block('human')

You can see how the memory is presented in the context window with `.compile()`, which uses the `prompt_template` to template the data: 

In [ ]:
memory.compile()

### Archival & Recall memory summaries
The agent also has access to external memories (stored in a database). There are two types of external memory: 
* *Archival memory*: Memories stored in a vector database that are either saved by the agent itself, or loaded in by the user
* *Recall memory*: The full conversational history of the agent

Both of these memories stores can be queried by the agent for RAG. To ensure the agent knows that these external memories stores may have relevant information, the context window contains a summary of the number of rows in both archival and recall memory. 

In [ ]:
client.get_archival_memory_summary(agent_state.id)

In [ ]:
client.get_recall_memory_summary(agent_state.id)

You can also directly query the full conversational history: 

In [ ]:
client.get_messages(agent_state.id)

## Section 4: Modifying core memory 
The core memory can adapt over time as new information is provided about the human (or about the agent itself). Letta agents have the ability to adapt their memory by modifying their context window.  

In [ ]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "My name is actually Charles", 
    role = "user"
) 
nb_print(response.messages)

Now we can see the updated core memory: 

In [ ]:
client.get_core_memory(agent_state.id).get_block("human")

## Section 5: Modifying archival memory 
The agent can also use the archival memory store to save memories. Since archival memory is a vector DB, we can also directly insert in memories - this can be useful if you have external data sources that you want the agent to be able to connect to via memory. 

First, lets trigger the agent to write an archival memory: 

In [ ]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "Save the information that 'bob loves cats' to archival", 
    role = "user"
) 
nb_print(response.messages)

We can also insert an archival memory manually: 

In [ ]:
client.insert_archival_memory(
    agent_state.id, 
    "Bob's loves boston terriers"
)

Now, we can have the agent run RAG to answer a specific question: 

In [ ]:
response = client.send_message(
    agent_id=agent_state.id, 
    role="user", 
    message="What animals do I like? Search archival."
)
nb_print(response.messages)

## Section 6: Demonstrating Memory Categorization and Prioritization
In this section, we will demonstrate how to categorize and prioritize memories using the new memory management system.

In [ ]:
from helper import Memory, MemoryManager

memory_manager = MemoryManager()

memory1 = Memory("Learned about AI", 10)
memory2 = Memory("Had lunch", 5)
memory3 = Memory("Went for a walk", 7)

memory_manager.add_memory(memory1, "short_term")
memory_manager.add_memory(memory2, "long_term")
memory_manager.add_memory(memory3, "episodic")

memory_manager.prioritize_memories()
print("Prioritized Memories:", memory_manager.get_memories("short_term"))

## Section 7: Demonstrating Memory Decay Mechanism
In this section, we will demonstrate how the decay mechanism works to gradually forget less important memories over time.

In [ ]:
import time

memory_manager.decay_memories()
print("Memories after decay:", memory_manager.get_memories("short_term"))